# Insights Into Employee Attrition: Predictive Modeling With HR Data


# Introduction

## Data Overview & Relevance
We have chosen to work with the HR Employee Attrition Dataset, which can be accessed [here](https://www.kaggle.com/datasets/rishikeshkonapure/hr-analytics-prediction/).

This dataset is concerned with factors contributing to employee attrition. In the context of this dataset, attrition refers to an employee voluntarily leaving their place of work. There are a multitude of reasons an employee may attrit; according to a study by Firth et al., the most significant factors are job dissatisfaction, feelings of stress precipitated by the presence of job stressors, and lack of commitment to the organization (Firth et al., 2004).

Employee attrition impedes business efficiency, disrupts current employees' ability to function, and incurs additional costs in hiring and training new employees. Predicting employee attrition is thus a valuable tool for any company. Understanding the factors that contribute to employee attrition helps companies identify and address employee retention issues, and use the findings of the analysis to inform their hiring process. It also benefits employees, by fostering an environment that prioritizes employment longevity (Boushay & Glynn, 2012). Boushay and Glynn delve deeper into the mutual advantages for both employers and employees that arise from mitigating attrition, as discussed in their 2012 paper.

**Research Question:** What combination of environmental and demographic variables can predict the binary response variable, Attrition, most accurately?<br>
<br>
**Objective:** Identify an optimal prediction model that will offer valuable insights to employers during the employee selection process, and minimize the likelihood of future employee attrition.<br>
<br>
To achieve our stated objective, we plan to develop and compare multiple models, including a full model encompassing all available variables and selective models that includes a subset of relevant variables.

## Data Description and Acknowledgements
This dataset belongs to Rushikesh Konapure; the original source of the data is IBM.

This dataset contains 35 variables and 1,470 observations. Each row represents a single employee from the same company.
The variable metadata can be accessed [here](http://inseaddataanalytics.github.io/INSEADAnalytics/groupprojects/January2018FBL/IBM_Attrition_VSS.html).

A comprehensive description of the data post-processing is provided under Clean & Tidy Data Characterization.

In [ ]:
suppressPackageStartupMessages({

library(dplyr)
library(caret)
library(reshape2)
library(GGally)
library(tidyverse)
library(ggplot2)
library(car)
library(tidyverse)
library(tidymodels)
library(broom)
library(glmnet)
library(leaps)
library(faraway)
library(mltools)
library(caret)
library(pROC)
library(randomForest)
    
})

In [ ]:
url <- "https://raw.githubusercontent.com/vanessavalentine/Stat301_Group-9_Vanessa/main/HR-Employee-Attrition.csv"
data <- read.csv(url)
head(data, 3)

In [ ]:
# Original Data variables
str(data)

# Methods and Results
## Data cleaning and wrangling
To prime our data for subsequent analytical tasks, we performed the data cleaning & wrangling procedures outlined below.

#### 1. Check for missing data
Our data contained no missing values to be imputed.
   
#### 2. Excluding Irrelevant Variables
* The variable **'EmployeeNumber'** is not pertinent to our analysis and will be excluded.

* The variables **'DailyRate'** , **MonthlyRate** and **HourlyRate** will be omitted, as they are redundant given the presence of **'MonthlyIncome'**.

* The columns **EmployeeCount**, **Over18**, and **StandardHours** hold constant values for every observation in the dataset. Features that do not vary across observations are not useful for further analysis. Consequently, they must be removed from the dataset.

#### 3. Adjusting Ordinal Values
Certain categorical variables are currently encoded as ordinal numbers (Likert Scale), and should be converted to their corresponding textual descriptors. This will enhance the interpretability of visual representations, and ensure clarity when we proceed to create dummy variables.

#### 4. Recategorizing Variables
* **StockOptionLevel:** ranging from 0 to 3, this variable lacks specific explanations. To enhance interpretability, we'll reclassify it: 0 as 'No' (no stock options granted), and 1 to 3 as 'Yes' (indicating stock options granted). It will also be renamed to **StockOption**.<br/>

* **Age:** previously, the Age variable was continuous. We will modify it into 4 categories: 18-29, 30-39, 40-49, and 50 and above.<br/>

#### 5. Transforming Categorical Variables
All character variables in our dataset will be transformed to factors, ensuring that categorical data are properly recognized for analysis. This includes the response variable 'Attrition', since its possible values are 'Yes' or 'No'.

Ordered levels will be assigned to variables measured on a Likert scale (e.g. Education, Job Satisfaction, Work-Life Balance), which allows for more meaningful statistical comparisons and visualizations that respect the inherent order of these ratings.

#### 6. Renaming an Undescribed Category

**JobLevel:** Job level is a categorical ordinal that ranges from [1,5]. However, the metadata doesn't specify what each level means. We will rename the categories as follows:<br>
* 1 = Entry<br>
* 2 = Intermediate<br>
* 3 = Experienced<br>
* 4 = Advanced<br>
* 5 = Expert<br>

This methodology is adopted based on [reference material](https://shr.ucsc.edu/compensation/career-tracks-classification-system/categories-and-levels-6-29-16.pdf) from the University of California, Santa Cruz.

In [ ]:
# Step 1)
sum(is.na(data))

In [ ]:
# Step 2)
for (col in names(data)) {
    if (length(unique(data[[col]])) == 1) {
        cat("Column", col, "has identical values. \n")
    }
}

In [ ]:
# Remove all the variables identified above
data <- data %>% select(-EmployeeNumber, -DailyRate, -StandardHours,
                        -Over18, -MonthlyRate, -HourlyRate, -EmployeeCount) 

In [ ]:
# Step 3)
data <- data %>%
  mutate(Education = case_when(
    Education == 1 ~ "below-college",
    Education == 2 ~ "college",
    Education == 3 ~ "bachelor",
    Education == 4 ~ "master", 
    Education == 5 ~ "PHD"
  ))

data <- data %>%
  mutate(EnvironmentSatisfaction = case_when(
    EnvironmentSatisfaction == 1 ~ "Low",
    EnvironmentSatisfaction == 2 ~ "Medium",
    EnvironmentSatisfaction == 3 ~ "High",
    EnvironmentSatisfaction == 4 ~ "Very-High"
  ))

data <- data %>%
  mutate(JobInvolvement = case_when(
    JobInvolvement == 1 ~ "Low",
    JobInvolvement == 2 ~ "Medium",
    JobInvolvement == 3 ~ "High",
    JobInvolvement == 4 ~ "Very-High"
  ))

data <- data %>%
  mutate(JobSatisfaction = case_when(
    JobSatisfaction == 1 ~ "Low",
    JobSatisfaction == 2 ~ "Medium",
    JobSatisfaction == 3 ~ "High",
    JobSatisfaction == 4 ~ "Very-High"
  ))

data <- data %>%
  mutate(PerformanceRating = case_when(
    PerformanceRating == 1 ~ "Low",
    PerformanceRating == 2 ~ "Good",
    PerformanceRating == 3 ~ "Excellent",
    PerformanceRating == 4 ~ "Outstanding"
  ))

data <- data %>%
  mutate(RelationshipSatisfaction = case_when(
    RelationshipSatisfaction == 1 ~ "Low",
    RelationshipSatisfaction == 2 ~ "Good",
    RelationshipSatisfaction == 3 ~ "Excellent",
    RelationshipSatisfaction == 4 ~ "Outstanding"
  ))

data <- data %>%
  mutate(WorkLifeBalance = case_when(
    WorkLifeBalance == 1 ~ "Bad",
    WorkLifeBalance == 2 ~ "Good",
    WorkLifeBalance == 3 ~ "Better",
    WorkLifeBalance == 4 ~ "Best"
  ))

data <- data %>%
  mutate(JobLevel = case_when(
    JobLevel == 1 ~ "Entry",
    JobLevel == 2 ~ "Intermediate",
    JobLevel == 3 ~ "Experienced",
    JobLevel == 4 ~ "Advanced", 
    JobLevel == 5 ~ "Expert"
  ))


In [ ]:
# Step 4)
data <- data %>%
  mutate(StockOption = case_when(
      StockOptionLevel == 0 ~ "No", 
      StockOptionLevel > 0 ~ "Yes"
  )) %>% select(-StockOptionLevel)


In [ ]:
data <- data %>%
  mutate(across(where(is.character), as.factor))

In [ ]:
# Steps 5) & 6)
data <- data %>%
  mutate(
       # Make the ordinal / Likert scale as levels
         Education = factor(Education, levels = c("below-college", "college", "bachelor", "master", "PHD"), 
                            ordered = TRUE),
         EnvironmentSatisfaction = factor(EnvironmentSatisfaction, levels = c("Low", "Medium", "High", "Very-High")
                                          , ordered = TRUE),
         JobInvolvement = factor(JobInvolvement, levels = c("Low", "Medium", "High", "Very-High")
                                          , ordered = TRUE),
         JobSatisfaction = factor(JobSatisfaction, levels = c("Low", "Medium", "High", "Very-High")
                                          , ordered = TRUE),
         PerformanceRating = factor(PerformanceRating, levels = c("Low", "Good", "Excellent", "Outstanding")
                                          , ordered = TRUE),
         RelationshipSatisfaction = factor(RelationshipSatisfaction, levels = c("Low", "Good", "Excellent", "Outstanding")
                                          , ordered = TRUE),
         WorkLifeBalance = factor(WorkLifeBalance, levels = c("Bad", "Good", "Better", "Best")
                                          , ordered = TRUE), 
         # Level of the Job, where 1 is the lowest level and 5 is the highest
         JobLevel = factor(JobLevel, levels = c("Entry", "Intermediate", "Experienced", "Advanced", "Expert")
                                          , ordered = TRUE),
         # Travel frequency for job purposes
         BusinessTravel = factor(BusinessTravel, levels = c("Non-Travel", "Travel_Rarely", "Travel_Frequently")
                                          , ordered = TRUE)
        )


In [ ]:
head(data, 3)

### Clean & Tidy Data Characterization

After cleaning & wrangling, our data consists of 28 variables (1 response variable and 27 independent variables), and 1,470 data points.

The variables are characterized in more detail in the tables below.

#### Continuous & Discrete Integer Variables

| Name                     | Type     | Category | Range                                                |
| ------------------------ | -------- | -------- | -------------                                               |
| DistanceFromHome         | Integer  | Continuous | 1 to 29                                                     |
| MonthlyIncome            | Integer  | Continuous | 1009 to 19999                                               |
| NumCompaniesWorked       | Integer  | Continuous | 0 to 9                                                      |
| PercentSalaryHike        | Integer  | Continuous | 11 to 25                                                    |
| TotalWorkingYears        | Integer  | Continuous | 0 to 40                                                     |
| TrainingTimesLastYear    | Integer  | Discrete | 0 to 6                                                       |
| YearsAtCompany           | Integer  | Continuous | 0 to 40                                                     |
| YearsInCurrentRole       | Integer  | Continuous | 0 to 18                                                     |
| YearsSinceLastPromotion  | Integer  | Continuous | 0 to 15                                                     |
| YearsWithCurrManager     | Integer  | Continuous | 0 to 17                                                     |
| Age                      | Integer  | Continuous | -                                                           |

#### Categorical & Ordinal Variables

| Name                     | Type     | Category | Cardinality | Unique Values                                 |
| ------------------------ | -------- | -------- | ----------- | -----------------------------------------------------------------------------|
| Attrition                | Categorical | Str   | 2           | Yes, No                                                                      |
| Department               | Character| Categorical | 3           | Sales, Research & Development, Human Resources                               |
| EducationField           | Character| Categorical | 6           | Life Sciences, Medical, Marketing, Technical Degree, Human Resources, Other   |
| Gender                   | Character| Categorical | 2           | Female, Male                                                                  |
| JobRole                  | Character| Categorical | 9           | Sales Executive, Research Scientist, Laboratory Technician, Manufacturing Director, Healthcare Representative, Manager, Sales Representative, Research Director, Human Resources |
| MaritalStatus            | Character| Categorical | 3           | Divorced, Married, Single                                                    |
| OverTime                 | Character| Categorical | 2           | Yes, No                                                                       |
| StockOption              | Character| Categorical | 2           | Yes, No                                                                       |
| BusinessTravel           | Character| Categorical | 3           | Non-Travel, Travel_Rarely, Travel_Frequently                                 |
| Education                | Integer  | Ordinal  | 5           | below-college, college, bachelor, master, PHD                                 |
| EnvironmentSatisfaction  | Integer  | Ordinal  | 4           | Low, Medium, High, Very-High                                                 |
| JobInvolvement           | Integer  | Ordinal  | 4           | Low, Medium, High, Very-High                                                 |
| JobLevel                 | Integer  | Ordinal  | 5           | Entry, Intermediate, Experienced, Advanced, Expert                            |
| JobSatisfaction          | Integer  | Ordinal  | 4           | Low, Medium, High, Very-High                                                 |
| PerformanceRating        | Integer  | Ordinal  | 4           | Low, Good, Excellent, Outstanding                                            |
| RelationshipSatisfaction | Integer  | Ordinal  | 4           | Low, Good, Excellent, Outstanding                                            |
| WorkLifeBalance          | Integer  | Ordinal  | 4           | Bad, Good, Better, Best                                                       |

## a) Exploratory Data Analysis

In [ ]:
options(repr.plot.width=5, repr.plot.height=5)

data_continuous <- data %>% 
    select(DistanceFromHome, MonthlyIncome, NumCompaniesWorked,
    PercentSalaryHike, TotalWorkingYears, TrainingTimesLastYear,
    YearsAtCompany, YearsInCurrentRole, YearsSinceLastPromotion,
    YearsWithCurrManager, Age)

correlation_matrix <- cor(data_continuous)

correlation_melted <- melt(correlation_matrix)

ggplot(data = correlation_melted, aes(x = Var1, y = Var2)) +
  geom_tile(aes(fill = value)) +
  scale_fill_gradient(low = "blue", high = "red") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  coord_fixed(ratio = 1)

In [ ]:
options(repr.plot.width=12, repr.plot.height=12)

data %>%
select(Attrition, DistanceFromHome, MonthlyIncome, NumCompaniesWorked,PercentSalaryHike,YearsAtCompany, YearsSinceLastPromotion, Age, JobSatisfaction,WorkLifeBalance, RelationshipSatisfaction,OverTime) %>%
    ggpairs(aes(color = Attrition), lower = "blank", legend = 1,
    diag = list(continuous = wrap("densityDiag", alpha = 0.5))) +
    ggtitle("Final Visualization: Correlation and Distribution of Selected Variables by Attrition") +
    theme(legend.position = "bottom")

#### Brief description of the chosen continuous variables and their plots:

* **DistanceFromHome:** Slightly higher median values for those who left the company (Yes Attrition), which might imply that longer commutes may correlate with a higher likelihood of leaving the job. 
* **MonthlyIncome:** Hourly rate does not show a clear distinction between those who stayed and those who left. 
* **NumCompaniesWorked:** Employees who have worked at more companies tend to attrit more, suggesting a potentially persistent pattern of jobhopping.
* **PercentSalaryHike:** Salary hikes seems to be constant between those who attrited and those who didn't. 
* **TotalWorkingYears:** Employees with fewer total working years have a higher tendency to leave, potentially reflecting early-career job exploration. 
* **YearsAtCompany:** Those who left generally have fewer years at the company, suggesting that those who have stayed longer are less likely to seek employment elsewhere.
* **YearsSinceLastPromotion:** A higher concentration of individuals with a recent promotion in the group that stayed, hinting that promotions may contribute to employee retention. 
* **Age:** A younger workforce appears to be leaving more, which might relate to career exploration or lesser attachment to the company. 
* **JobSatisfaction:** Lower job satisfaction levels are visibly associated with higher attrition, suggesting the importance of job satisfaction in employee retention.
* **WorkLifeBalance:** Poor work-life balance is subtly associated with higher attrition, suggesting it plays a role in attrition. 
* **RelationshipSatisfaction:** Lower levels of relationship satisfaction are seen in those who left, which could contribute to an uncomfortable work environment leading to higher attrition. 
* **OverTime:** A clear pattern where employees who have more overtime are more likely to leave, which might point towards work-life imbalance as reasons for attrition.

In [ ]:
# Categorical variables with factor levels
data_categorical <- data %>%
  select_if(function(x) is.factor(x))

factor_vars <- names(data_categorical)[sapply(data_categorical, is.factor)]

# Create a function to generate and display bar plots
generate_and_display_bar_plot <- function(var_name) {
  melted_data <- tidyr::gather(data_categorical, key = "variable", value = "value", var_name)
    g <- ggplot(melted_data, aes(x = value)) +
    geom_bar() +
    facet_wrap(~ variable, scales = "free") +
    theme_minimal() +
    labs(title = paste("Barplot of Counts for", var_name),
         x = "Value",
         y = "Count")
    print(g)
}
            
options(repr.plot.width = 5, repr.plot.height = 3) 

# Generate and display separate bar plots for each factor variable
for (var in factor_vars) {
  generate_and_display_bar_plot(var)
}

###  Insight from the visualization

#### Brief descriptions of the distributions for categorical variables:**

- **Attrition:** many more employees attritted than not; 1233:237
- **BusinessTravel:** most employees do not have to travel frequently, but if they do have to travel, it is freuqent
- **Department:** Biggest department is R&D, then HR, then sales
- **Education:** most employees have a bachelors or masters as their highest level of education
- **EducationField:** most employees studied a non-listed field, or marketing
- **EnvironmentSatisfaction:** most employees have high or very high environment satisfaction
- **Gender:** there are more male than female employees, but not by a large margin; 588:882
- **JobInvolvement:** most employees rate themselves as having a highly involved job
- **JobLevel:** most employees have a level 1- or 2-ranked job
- **JobRole:** the most common job roles are research director, Sales rep, and lab tech; the least common was research scientist
- **JobSatisfaction:** most employees rate their job satisfaction as High or Very High
- **MaritalStatus:** most employees are married; second most frequent status is divorced; third is single
- **OverTime:** most employees work overtime; 1054:416
- **RelationshipSatisfacton:** most employees rate their relationship satisfaction as High or Very High
- **StockOptionLevel:** most employees are given stock option level 1 or 2
- **WorkLifeBalance:** most employees rate their work-life balance as High
- **StockOption:** More employees get stock option

## b) Methods: Plan

To address our stated research question, we will employ two modeling techniques: logistic regression and random forest. Because the data contains 26 potential predictor variables, the use of LASSO for feature selection is a critical component of our methodology. By implementing LASSO, we can efficiently identify and retain only the most significant variables. The selection of the most appropriate model will be based on several criteria, including the analysis of ROC curves, prediction accuracy, and AUC. These metrics will guide us in choosing the model that best fits our data and provides the most reliable predictions.

Our exploratory data analysis revealed the presence of multicollinearity and non-normality. Multicollinearity could lead to unreliable parameter estimates in the models. While LASSO can help in reducing the effects of multicollinearity by shrinking coefficients, it may not be entirely effective if multicollinearity is severe. Non-normality, on the other hand, violates the assumption of normality. As a result, the efficiency and accuracy of LASSO regression can be compromised when dealing with non-normally distributed data. To address this, we might employ data transformations (log transform and square root transform).

Train and test data will be established using a 70/30 split. Our approach will initially employ LASSO for feature selection, followed by the development of various modeling techniques. While LASSO might be sufficient enough to mitigate multicollinearity during feature selection, it doesn't fully address issues related to strong multicollinearity and non-normality; a thorough approach, including extra preliminary steps for treating outliers and implementing methods to manage multicollinearity, is essential to ensure optimal performance of our models.

In [ ]:
# We remove YearsWithCurrManager because it highly correlated with a lot of variables
data_clean <- data %>% select(Attrition, DistanceFromHome, MonthlyIncome, NumCompaniesWorked, 
                      PercentSalaryHike, TotalWorkingYears, TrainingTimesLastYear,
                      YearsAtCompany, YearsInCurrentRole, YearsSinceLastPromotion,
                       Age, BusinessTravel, Department, Education, 
                      EducationField, EnvironmentSatisfaction, Gender, JobInvolvement,
                      JobLevel, JobRole, JobSatisfaction, MaritalStatus, OverTime,
                      PerformanceRating, RelationshipSatisfaction, WorkLifeBalance, StockOption)
# make attrition as numeric so that we can use it for lasso
data_clean$Attrition <- as.numeric(data_clean$Attrition)-1


In [ ]:
#perform transform to normalize data
data_clean <- data_clean %>%
                       mutate(
                         Age = log(Age),
                         MonthlyIncome = log(MonthlyIncome),
                         TotalWorkingYears = sqrt(TotalWorkingYears),
                         YearsInCurrentRole = sqrt(YearsInCurrentRole), 
                         YearsSinceLastPromotion = sqrt(YearsSinceLastPromotion),
                         PercentSalaryHike = log(PercentSalaryHike)
                       )

In [ ]:
# train test split 
set.seed(4321) # Do not change this

attrition_split <- initial_split(data_clean, prop = 0.7, strata = Attrition)
attrition_selection <- training(attrition_split)
attrition_inference <- testing(attrition_split)

# Create model matrix for the predictors with one-hot encoding (dummy variable)
predictors <- model.matrix(~ . -1, data = attrition_selection %>% select(-Attrition))

In [ ]:
model_matrix_X_train <- model.matrix(Attrition ~ ., attrition_selection)[, -1]
matrix_Y_train <- as.matrix(attrition_selection$Attrition, ncol = 1)

model_matrix_X_test <- model.matrix(Attrition ~ ., attrition_inference)[, -1]
matrix_Y_test<- as.matrix(attrition_inference$Attrition, ncol = 1)

In [ ]:
# ordinary logistic model
set.seed(4321)
attrition_logistic_model <- 
    glm(
        formula = Attrition ~ .,
        data = attrition_selection,
        family = binomial)
ordinary_predicted_probabilities <- predict(attrition_logistic_model, newdata = attrition_inference, type = "response")


ordinary_predicted_classes <- ifelse(ordinary_predicted_probabilities >= 0.5, 1, 0)

ordinary_conf_mat <- confusionMatrix(data = as.factor(ordinary_predicted_classes),
                reference = as.factor(attrition_inference$Attrition),
                positive ='1')
ordinary_conf_mat
ordinary_accuracy <- ordinary_conf_mat$overall["Accuracy"]
ordinary_roc <- roc(response = attrition_inference$Attrition, predictor = ordinary_predicted_probabilities)
ordinary_auc <- auc(ordinary_roc)
ordinary_aic <- AIC(attrition_logistic_model)

In [ ]:
# use lasso to select variables
set.seed(4321)

attrition_cv_lambda_LASSO <-
    cv.glmnet(model_matrix_X_train, 
              matrix_Y_train, 
              alpha=1,
              family = "binomial",
              type.measure = "auc",
              nfolds = 5 
             )

attrition_cv_lambda_LASSO

attrition_lambda_1se_AUC_LASSO <- round(attrition_cv_lambda_LASSO$lambda.1se, 4)

In [ ]:
# extract coefficient for lasso logistic regression
set.seed(4321)
attrition_LASSO_1se_AUC <- glmnet(
  x = model_matrix_X_train, y = matrix_Y_train,
  alpha = 1,
  family = "binomial",
  lambda = attrition_lambda_1se_AUC_LASSO
)

attrition_LASSO_1se_AUC
coef(attrition_LASSO_1se_AUC)

##### Based on LASSO, we will remove "PerformanceRating", "EducationField", "EducationField", "Education", "YearsSinceLastPromotion", "YearsAtCompany", "TotalWorkingYears", "TrainingTimesLastYear", "PercentSalaryHike" because its coefficent have been shrunk to zero. This means that it is not significantly impactful in predicting attrition.

In [ ]:
set.seed(4321) # Do not change this

attrition_split <- initial_split(data_clean, prop = 0.7, strata = Attrition)
attrition_selection <- training(attrition_split)
attrition_inference <- testing(attrition_split)


attrition_selection <- attrition_selection %>% select(-PerformanceRating, -EducationField, -Education, -YearsSinceLastPromotion,
                                                      -YearsAtCompany, -TrainingTimesLastYear,-PercentSalaryHike )
attrition_inference <- attrition_inference %>% select(-PerformanceRating, -EducationField, -Education, -YearsSinceLastPromotion,
                                                      -YearsAtCompany, -TrainingTimesLastYear,-PercentSalaryHike )

In [ ]:
# ordinary logistic model
log_attrition_lasso <- 
    glm(
        formula = Attrition ~ .,
        data = attrition_selection,
        family = binomial)

summary(log_attrition_lasso)

log_attrition_lasso_result  <-
    tidy(log_attrition_lasso) %>% 
    mutate(exp.estimate = exp(estimate)) %>% 
    mutate_if(is.numeric, round, 3)

In [ ]:
# lasso logistic model
lasso_predicted_probabilities <- predict(log_attrition_lasso, newdata = attrition_inference, type = "response")

lasso_predicted_classes <- ifelse(lasso_predicted_probabilities >= 0.5, 1, 0)

lasso_conf_mat<- confusionMatrix(data = as.factor(lasso_predicted_classes),
                reference = as.factor(attrition_inference$Attrition),
                positive ='1')
lasso_conf_mat

lasso_accuracy <-lasso_conf_mat$overall["Accuracy"]
lasso_roc <- roc(response = attrition_inference$Attrition, predictor = lasso_predicted_probabilities)
lasso_auc <- auc(lasso_roc)
lasso_aic <- AIC(log_attrition_lasso)

In [ ]:
# Random Forest 
set.seed(4321)
attrition_selection$Attrition <- as.factor(attrition_selection$Attrition)
attrition_inference$Attrition <- as.factor(attrition_inference$Attrition)

random_forest_model <- randomForest(Attrition ~ ., data = attrition_selection, ntree = 500)
rf_predictions <- predict(random_forest_model, newdata = attrition_inference)

rf_conf_matrix <- confusionMatrix(rf_predictions, attrition_inference$Attrition)
print(rf_conf_matrix)

rf_probabilities <- predict(random_forest_model, newdata = attrition_inference, type = "prob")
rf_prob_of_positive_class <- rf_probabilities[, "1"]
rf_accuracy <-rf_conf_matrix$overall["Accuracy"]
rf_roc <- roc(as.numeric(attrition_inference$Attrition), rf_prob_of_positive_class)
rf_auc <- auc(rf_roc)

In [ ]:
models_comparison <- data.frame(
  Model = c("Ordinary Logistic Regression", "LASSO Logistic Regression", "Random Forest"),
  AIC = c(AIC(attrition_logistic_model), AIC(log_attrition_lasso), NA),  # NA for Random Forest
  AUC = c(ordinary_auc, lasso_auc, rf_auc),
  Prediction_Accuracy = c(ordinary_accuracy, lasso_accuracy, rf_accuracy)
)
models_comparison

In [ ]:
options(repr.plot.width=10, repr.plot.height=7)
plot(ordinary_roc, main = "ROC Curves Comparison", col = "blue", lwd = 2)
plot(lasso_roc, col = "red", add = TRUE, lwd = 2)
plot(rf_roc, col = "green", add = TRUE, lwd = 2)

legend("bottomright", legend = c("Ordinary Logistic", "LASSO Logistic", "Random Forest"),
       col = c("blue", "red", "green"), lwd = 2)

## Discussion

Our comparative study of classification models utilized ordinary logistic regression, logistic regression with LASSO feature selection, and a random forest model. The LASSO model performed the best with respect to prediction accuracy, achieving a score of 90.50%, followed by the ordinary logistic regression at 88.91%, while the random forest model scored 85.97% accuracy. 

Assessing the models' discriminative capabilities with AUC revealed that the ordinary logistic regression model leads with an AUC of 0.8789, indicating superior performance in differentiating between instances of attrition and non-attrition. The LASSO model shows slightly reduced effectiveness with an AUC of 0.8651, while the random forest model ranks third with an AUC of 0.8424. 

The ROC indicates that the ordinary logistic regression model is the most proficient model, followed by the LASSO model, with the random forest model being the least capable.

Considering model complexity and goodness-of-fit as indicated by the AIC, the LASSO model appears to offer the most parsimonious fit with the lowest AIC value, a metric not applicable to the non-parametric random forest model. While LASSO logistic regression offers the best balance between accuracy and model simplicity, ordinary logistic regression exhibits the highest discriminative ability.

In the LASSO model, we used the variables
- `DistanceFromHome`
- `MonthlyIncome`
- `NumCompaniesWorked`
- `YearsInCurrentRole`
- `Age`
- `DepartmentResearch & Development`
- `DepartmentSales`
- `MaritalStatusSingle`
- `OverTimeYes`
- `StockOptionYes`

and some levels in our created dummy variables: 
- `BusinessTravel.L`
- `EnvironmentSatisfaction.L`
- `JobInvolvement.L`, `JobLevel.Q`
- `JobLevel.C`
- `JobLevel^4`
- `JobRoleLaboratory Technician`
- `JobRoleResearch Director`
- `RelationshipSatisfaction.L`
- `RelationshipSatisfaction.Q`
- `WorkLifeBalance.Q`

Most of these predictor variables aligned with our expectations with respect to their significance in predicting employee attrition, with the exception of the two job roles, laboratory technician and research director. Before fitting the model, we believed that job level could affect attrition rate, but not job role. Our results indicate possible problems with these two job roles, since all other job positions are unrelated. The achieved prediction accuracy exceeded our initial target. Attaining a prediction accuracy over 95% is not practical or easy, so our goal from the outset was to achieve a prediction accuracy of 90%.

The performance of our model can be further enhanced. Only a subset of variables are eliminated via LASSO; the resulting model is complicated and prone to overfitting. Leave-one-out or K-fold Cross-validation are methods that could improve variable selection performance. Though we did incorporate dummy variables appropriately, interaction terms were not taken into account. Given the quantity of predictors that are similar or related, interaction could have significant effects on model accuracy.

The research objective of this project has many practical applications. Modeling the attrition of employees against multiple factors allows one to examine the probability of attrition across various work environments. This could offer valuable insights to managers or leaders on methods to minimize costly employee attrition and attract high-quality talent from the job market. Additionally, our approach can be extended to many domains, such as organizations or educational institutions, to explore the impact of factors on students or participants.

## References

Firth, L., Mellor, D., Moore, K. A., & Loquet, C. (2004). How Can Managers Reduce Employee Intention to Quit? *Journal of Managerial Psychology*, 19(2), 170–187. https://doi.org/10.1108/02683940410526127

Boushey, H., & Glynn, J. (2012). There Are Significant Business Costs to Replacing Employees. *Center for American Progress*, 1–9. http://cdn.americanprogress.org/wp-content/uploads/2012/11/CostofTurnover.pdf